## 准备

In [1]:
from typing import List

## 过程

### 1

第一想法就是，将两数组加在一起，排序，取出中间的数。

In [3]:
class Solution:
    def findMedianSortedArrays(self, nums1: List[int], nums2: List[int]) -> float:
        nums = nums1 + nums2
        nums.sort()

        length = len(nums)
        if length % 2 == 0:
            return (nums[length//2-1] + nums[length//2]) / 2
        else:
            return nums[length//2]

执行用时 :72 ms, 在所有 Python3 提交中击败了92.80%的用户

内存消耗 :13.3 MB, 在所有 Python3 提交中击败了66.49%的用户

时间复杂度：O(m + n) + O((m + n)log(m + n))    (合并数组 + 快速排序)

空间复杂度：O(m + n)

PS：

试试

就过了?

考虑到题目中所说难度`困难`。。。

还要求时间复杂度`O(log(m + n))`，应该是指快速排序的开销。

也没用上`有序`的条件。

看一下题解。。。

## 2

考虑到两个数组有序，直接拼接到一个新的数组里。

如果新数组已经有了(m+n)/2个数，那么就可以停止了，取最后1个或2个数作中位数。

In [4]:
class Solution:
    def findMedianSortedArrays(self, nums1: List[int], nums2: List[int]) -> float:
        length = len(nums1) + len(nums2)
        cycle = length // 2 - 1
        
        # 取出2数组中前面多余的元素
        for i in range(cycle):
            if nums2 == [] or (nums1 != [] and nums2 != [] and nums1[0] < nums2[0]):
                nums1.pop(0)
            else:
                nums2.pop(0)

        # 如果2个数组总共只有1个元素，直接返回
        if length == 1:
            return nums1[0] if nums1 != [] else nums2[0]
        
        # 元素多于1个的情况
        result = []
        for i in range(2):
            if nums2 == [] or (nums1 != [] and nums2 != [] and nums1[0] < nums2[0]):
                result.append(nums1.pop(0))
            else:
                result.append(nums2.pop(0))
        
        if length % 2 == 0:
            return sum(result) / 2
        else:
            return result[1]
            

nums_list = [
    ([1, 3], [2]),
    ([1, 2], [3, 4]),
    ([], [1]),
    ([2], [])
    ]
for nums1, nums2 in nums_list:
    print(Solution().findMedianSortedArrays(nums1, nums2))

2
2.5
1
2


执行用时 :76 ms, 在所有 Python3 提交中击败了86.94%的用户

内存消耗 :13.1 MB, 在所有 Python3 提交中击败了96.86%的用户

时间复杂度：O((m + n) / 2)

空间复杂度：O(1)

en,好像差不多。

其实好像也不用pop操作，记录两数组当前位置即可。

## 题解

凑凑答案...

原来时间复杂度log是这么来的。

1.在两数组之间二分法查找到使其满足以下条件即可：
- len(left_part)=len(right_part)
- max(left_part)≤min(right_part)

其中 $j = \frac{m + n + 1}{2} - i$

2.在循环搜索中，我们只会遇到三种情况：

- (j = 0 or i = m or B[j - 1] ≤ A[i] 或是 (i = 0 or j = n or A[i-1] ≤ B[j])，这意味着 ii 是完美的，我们可以停止搜索。
- j > 0 and i < m and B[j - 1] > A[i] 这意味着 ii 太小，我们必须增大它。
- i > 0 and j < n and A[i - 1] > B[j] 这意味着 ii 太大，我们必须减小它。

3.当找到i时，中位数为：

- $\max(\text{A}[i-1], \text{B}[j-1])$, 当 m+n 为奇数时
- $\frac{\max(\text{A}[i-1], \text{B}[j-1]) + \min(\text{A}[i], \text{B}[j])}{2}$, 当 m+n 为偶数时

注意：

- 需要检查临界值：A或B空、i = 0 or m、j = 0 or n。
- 奇数个元素的数组，如果割在一个数上，那么这个数即属于左边，也属于右边。

PS：

其中，`2.`中的条件可以进一步通过数学推导简化：

i < m ⟹  j > 0 以及 i > 0  ⟹  j < n 始终成立，这是因为：

- $m ≤ n, i < m ⟹ j = \frac{m+n+1}{2} - i > \frac{m+n+1}{2} - m ≥ \frac{2m+1}{2} - m ≥ 0$
- $m ≤, i > 0 ⟹ j = \frac{m+n+1}{2} - i < \frac{m+n+1}{2} ≤ \frac{2n+1}{2} ≤ n $

所以，在情况2和3中，我们不需要检查j > 0或j < n是否成立。

In [5]:
class Solution:
    def findMedianSortedArrays(self, nums1: List[int], nums2: List[int]) -> float:
        A, B = nums1, nums2
        m, n = len(A), len(B)
        if m > n:   # 将对称问题约束简化情况
            A, B, m, n = B, A, n, m
        if n == 0:
            raise ValueError

        imin, imax, half_len = 0, m, (m + n + 1) // 2
        while imin <= imax:
            i = (imin + imax) // 2
            j = half_len - i
            if i < m and B[j-1] > A[i]:
                # i is too small, must increase it
                imin = i + 1
            elif i > 0 and A[i-1] > B[j]:
                # i is too big, must decrease it
                imax = i - 1
            else:
                # i is perfect

                if i == 0: max_of_left = B[j-1]
                elif j == 0: max_of_left = A[i-1]
                else: max_of_left = max(A[i-1], B[j-1])

                if (m + n) % 2 == 1:
                    return max_of_left

                if i == m: min_of_right = B[j]
                elif j == n: min_of_right = A[i]
                else: min_of_right = min(A[i], B[j])

                return (max_of_left + min_of_right) / 2.0
        
            
nums_list = [
    ([1, 3], [2]),
    ([1, 2], [3, 4]),
    ([1, 3, 5], [2, 4, 6]),
    ([1, 3, 5], [2, 4]),
    ([1, 100, 200, 300, 1000], [2, 10]),
    ([], [1]),
    ([2], [])
    ]
for nums1, nums2 in nums_list:
    print(Solution().findMedianSortedArrays(nums1, nums2))

2
2.5
3.5
3
100
1
2


时间复杂度：O(log(min(m, n)))

空间复杂度：O(1)

PS：尝试根据题解的解释写了半天，总在边界处理上解决不清楚。归根到底属于思路不够理清。

PS：注意到题解代码一开始通过判断长短，将一个对称的问题约束简化到一个固定情况。